In [91]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class TextCNN(nn.Module):
    
    def __init__(self):
        super(TextCNN, self).__init__()
        self.Total_filters = num_filters * len(filter_sizes)
        self.W = nn.Embedding(vocab_size, emb_dim)
        self.Weight = nn.Linear(self.Total_filters, num_classes)
        self.Bias = nn.Parameter(torch.ones([num_classes]))
        
        # nn.Conv2d = [input_channels, output_channels, (kernel_height, kernel_width)]
        self.Filter_list = nn.ModuleList([nn.Conv2d(1, num_filters, (filter_dim, emb_dim)) for filter_dim in filter_sizes])
        
    def forward(self,X):
        
        # input X는 look-up table을 할 수 있도록 각 토큰의 인덱스 형태로 제공받음 [batch, sequence_length]
        X = torch.LongTensor(X) #index이므로 long tensor로 변환해줌
        embedded = self.W(X) # [batch=1, sequence_length, emb_dim]
        
        # batch 및 channel 차원 추가 (conv2d layer의 input 형식에 맞게)
        embedded = embedded.unsqueeze(0)
        embedded = embedded.unsqueeze(0) #[batch=1, 1, sequence_length, emb_dim]
        
        # sequence_length
        sequence_length = embedded.shape[2]
        
        #convolution
        pooled = []
        for i in range(num_filters):
            conv = self.Filter_list[i]
            
            # Convolution 후 ReLU  
            out = F.relu(conv(embedded)) # out = [batch=1, 1, sequence_length - filter_size[i] + 1, 1]
            
            # Maxpooling for each Feature Map
            mp = nn.MaxPool2d((sequence_length - filter_sizes[i] + 1, 1)) # out = [batch=1, output_channels=3, 1, 1]
            
            # Append maxpooled units to pooled
            pooled.append(mp(out).squeeze())
        
        # Fully connected layer 
        fc = torch.cat(pooled, 0)
        out = self.Weight(fc) + self.Bias # out = [self.Total_filters, num_classes]
        
        return out

if __name__ == '__main__':
    
    emb_dim = 2
    num_filters = 3
    filter_sizes = [2,3,4]
    num_classes = 2
    
    # Corpus and Labeles
    sentences = ['i am happy guy', 'you are happy guy', 'it is really interesting', 'i love playing piano', 'she really love me', 
                 'it is so terrible', 'he is very ugly', 'she hate the man', 'i hate my ugly voice', 'it is not good']

    labels = [1,1,1,1,1,0,0,0,0,0] # 1 : Positive, 0: Negative
    
    # make inputs 
    word_list = " ".join(sentences).split()
    vocab = list(set(word_list))
    vocab_size = len(vocab)
    vocab_dict = {word:idx for idx, word in enumerate(vocab)}
    
    inputs = [[vocab_dict[i] for i in s.split()] for s in sentences]
    targets = torch.LongTensor([out for out in labels]) # label이므로 long tensor
    
    # get model
    model =  TextCNN()
    
    # optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # Training
    for epoch in range(5000):
        optimizer.zero_grad()
        loss = 0
        
        for batch in range(len(inputs)):
            
            output = model(inputs[batch])
            loss_tmp = criterion(output, targets[batch])
            loss += loss_tmp
            
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

Epoch: 1000 cost = 0.019409
Epoch: 2000 cost = 0.003515
Epoch: 3000 cost = 0.001238
Epoch: 4000 cost = 0.000545
Epoch: 5000 cost = 0.000268


In [95]:
# Test
test_text = 'i really love playing the piano'
tests = [np.asarray([vocab_dict[n] for n in test_text.split()])]
test_batch = torch.LongTensor(tests)[0]
out = model(test_batch).data

# Predict
predict = int(torch.argmax(out))
if predict == 0:
    print("'", test_text, "'"," is Bad Mean...")
else:
    print("'", test_text, "'"," is Good Mean!!")

' i really love playing the piano '  is Good Mean!!
